In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/estudiantes/estudiantes.csv


# Cargar los datos

In [2]:
df = pd.read_csv('/kaggle/input/estudiantes/estudiantes.csv')

df.head()
print(df.shape)

(395, 33)


# Preparar los datos

In [3]:
#Eliminar filas con valores nulos, esto es opcional ya que mejor lo remplazamos por ceros
#df= df.dropna()

#verificar que los tipos de datos de cada columna sean correcto
#tipos_de_dato = df.dtypes
#print(tipos_de_dato)

#Hacer que la codificacion este correcta para que no haya errores de caracteres.
# Convertir todas las celdas del DataFrame a UTF-8
#data_frame = data_frame.applymap(lambda x: x.encode('latin-1').decode('utf-8', 'ignore'))

# Encontrar registros con todos los valores duplicados
#duplicados_completos = df[df.duplicated(df.columns, keep=False)]

#Imprimir los registros completos duplicados si los hay
#if not duplicados_completos.empty:
  #  print("SI HAY:")
#else:
  #  print("No hay registros con todos los valores iguales en el DataFrame.")
    
# Eliminar registros duplicados del DataFrame
#data_frame_sin_duplicados = data_frame.drop_duplicates()
# Imprimir información sobre la cantidad de registros eliminados
#registros_duplicados_elim = len(data_frame) - len(data_frame_sin_duplicados)
#print("Se eliminaron {} registros duplicados del DataFrame.".format(registros_duplicados_elim))
#data_frame = data_frame_sin_duplicados

#Remplazar valores nulos de registros por 0
#data_frame.fillna(0, inplace=True)
#Verificar si hay alguna celda nula en el DataFrame
#hay_nulos = df.isnull().any()
#if hay_nulos.any():
 #   print("Hay al menos una celda nula en el DataFrame.")
#else:
 #   print("No hay celdas nulas en el DataFrame.")